## Back Test Data Generation

In [1]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from pycaret.classification import *

In [2]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    k_number = cfg['knn']['k_number']
    metric = cfg['knn']['metric']
    algorithm = cfg['knn']['algorithm']
    feature_1 = cfg['knn']['feature_1']
    feature_2 = cfg['knn']['feature_2']
    feature_3 = cfg['knn']['feature_3']
    feature_7 = cfg['knn']['feature_7']
    feature_8 = cfg['knn']['feature_8']   
    volume = cfg['feature']['volume']
    volume_size = cfg['sample']['volume_size']
    sample_count = cfg['sample']['count']
    candles = cfg['recommendation']['candle_count']
    pair = cfg['currency']['pair_2']

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_k_similar_candles(candle_id, dataset, k=k_number):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = metric, algorithm = algorithm) 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]][feature_1],
                            dataset.iloc[indices.flatten()[i]][feature_2],
                            dataset.iloc[indices.flatten()[i]][feature_3],
                            dataset.iloc[indices.flatten()[i]][feature_7],
                            dataset.iloc[indices.flatten()[i]][feature_8],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      feature_1,
                      feature_2,
                      feature_3,
                      feature_7,
                      feature_8,
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
filename = 'USD_CAD_H4.csv'
data = pd.read_csv(filename)

In [6]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_10',
       'SMA_20', 'F_SMA_5', 'F_SMA_10', 'F_SMA_20', 'O-H', 'O-L', 'O-C', 'H-L',
       'H-C', 'L-C', 'Direction', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5'],
      dtype='object')

In [7]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20,O-H,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-09-24,09:00:00,9,267,3,Thursday,2015-09-24T09:00:00.000000000Z,12505,1.33319,1.34168,1.33309,1.34064,1.3340,1.3318,1.3284,0.00664,0.00884,0.01224,-0.00849,0.00010,-0.00745,0.00859,0.00104,-0.00755,1,-0.00745,0.00849,0.00755,-0.00174,0.00103
1,2015-09-24,13:00:00,13,267,3,Thursday,2015-09-24T13:00:00.000000000Z,18586,1.34066,1.34142,1.33324,1.33546,1.3347,1.3326,1.3291,0.00076,0.00286,0.00636,-0.00076,0.00742,0.00520,0.00818,0.00596,-0.00222,0,0.00520,0.00076,0.00222,-0.00745,-0.00174
2,2015-09-24,17:00:00,17,267,3,Thursday,2015-09-24T17:00:00.000000000Z,10673,1.33546,1.33598,1.32914,1.33052,1.3343,1.3327,1.3295,-0.00378,-0.00218,0.00102,-0.00052,0.00632,0.00494,0.00684,0.00546,-0.00138,0,0.00494,0.00052,0.00138,0.00520,-0.00745
3,2015-09-24,21:00:00,21,267,3,Thursday,2015-09-24T21:00:00.000000000Z,7599,1.33142,1.33498,1.33101,1.33452,1.3349,1.3336,1.3300,-0.00038,0.00092,0.00452,-0.00356,0.00041,-0.00310,0.00397,0.00046,-0.00351,1,-0.00310,0.00356,0.00351,0.00494,0.00520
4,2015-09-25,01:00:00,1,268,4,Friday,2015-09-25T01:00:00.000000000Z,3582,1.33456,1.33500,1.33260,1.33264,1.3348,1.3337,1.3304,-0.00216,-0.00106,0.00224,-0.00044,0.00196,0.00192,0.00240,0.00236,-0.00004,0,0.00192,0.00044,0.00004,-0.00310,0.00494


## Selecting 1000 Random Candles

In [8]:
random_samples = data[data[volume] > volume_size].sample(n = sample_count)
Test_Candle = list(random_samples.index.values)

In [9]:
Test_Candle[0:10]

[24, 653, 9425, 9431, 4751, 1768, 1108, 2287, 2081, 4122]

In [10]:
#Test_Candle = np.random.randint(low=1, high=len(data)-40, size=500)


In [11]:
data.shape

(9651, 30)

In [12]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20,O-H,O-L,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-09-24,09:00:00,9,267,3,Thursday,2015-09-24T09:00:00.000000000Z,12505,1.33319,1.34168,1.33309,1.34064,1.3340,1.3318,1.3284,0.00664,0.00884,0.01224,-0.00849,0.00010,-0.00745,0.00859,0.00104,-0.00755,1,-0.00745,0.00849,0.00755,-0.00174,0.00103
1,2015-09-24,13:00:00,13,267,3,Thursday,2015-09-24T13:00:00.000000000Z,18586,1.34066,1.34142,1.33324,1.33546,1.3347,1.3326,1.3291,0.00076,0.00286,0.00636,-0.00076,0.00742,0.00520,0.00818,0.00596,-0.00222,0,0.00520,0.00076,0.00222,-0.00745,-0.00174


# <font color='red'>CANDLE LOOP</font>

In [13]:
%%time
#LR_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/FINAL_LR_25Nov2021_EURUSD')
#ET_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/FINAL_ET_25Nov2021_EURUSD')
KNN_FINAL_MODEL = load_model('FINAL_MODELS/USDCAD/02-12-2021_11-09_PM_knn_USDCAD')
#LIGHTGBM_FINAL_MODEL = load_model('FINAL_MODELS/EURUSD/01-12-2021_08-02_PM_lightgbm_EURUSD')
GBC_FINAL_MODEL = load_model('FINAL_MODELS/USDCAD/02-12-2021_11-37_PM_gbc_USDCAD')

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
#                              'Rec1':[],
#                              'Rec1_P':[],
#                              'Rec2':[],
#                              'Rec2_P':[],
#                              'Rec3':[],
#                              'Rec3_P':[],
 #                             'LR_Label':[],
 #                             'LR_Score':[],
 #                             'ET_Label':[],
 #                             'ET_Score':[],
                              'KNN_Label':[],
                              'KNN_Score':[],
 #                             'LIGHTGBM_Label':[],
 #                             'LIGHTGBM_Score':[],
                              'GBC_Label':[],
                              'GBC_Score':[],                              
                             })

for candle_no in Test_Candle:
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+candles]
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market= 1

    else:
        Current_Market = 0

    data = pd.read_csv(filename)
    data = data[[feature_1,
                 feature_2,
                 feature_3,
                 feature_7,
                 feature_8,
                ]]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    recs = []
    for indice in indices[1:5]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+candles]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade = 'BUY'
            recs.append((r2_score(Y, y_pred).round(2)*100))
        else:
            Predicted_Trade = 'SELL'
            recs.append((r2_score(Y, y_pred).round(2)*100) * -1)
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])
        
        
    data_unseen = pd.DataFrame ({
        'Rec1_Score': [recs[0]],
        'Rec2_Score': [recs[1]],
        'Rec3_Score': [recs[2]],
        'Rec4_Score': [recs[3]],
    })
    
#    lr_prediction = predict_model(LR_FINAL_MODEL, data=data_unseen)
#    LR_Label = lr_prediction['Label']
#    LR_Score = lr_prediction['Score']
    
#    et_prediction = predict_model(ET_FINAL_MODEL, data=data_unseen)
#    ET_Label = et_prediction['Label']
#    ET_Score = et_prediction['Score']
    
    knn_prediction = predict_model(KNN_FINAL_MODEL, data=data_unseen)
    KNN_Label = knn_prediction['Label']
    KNN_Score = knn_prediction['Score']
    
#    lightgbm_prediction = predict_model(LIGHTGBM_FINAL_MODEL, data=data_unseen)
#    LIGHTGBM_Label = lightgbm_prediction['Label']
#    LIGHTGBM_Score = lightgbm_prediction['Score']

    gbc_prediction = predict_model(GBC_FINAL_MODEL, data=data_unseen)
    GBC_Label = gbc_prediction['Label']
    GBC_Score = gbc_prediction['Score']    
    
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
#              'Rec1': predicted_output[0][0],
#              'Rec1_P': predicted_output[0][1],
#              'Rec2': predicted_output[1][0],
#              'Rec2_P': predicted_output[1][1],
#              'Rec3': predicted_output[2][0],
#              'Rec3_P': predicted_output[2][1],
  #            'LR_Label': LR_Label[0],
  #            'LR_Score': LR_Score[0],
  #            'ET_Label': ET_Label[0],
  #            'ET_Score': ET_Score[0],
              'KNN_Label': KNN_Label[0],
              'KNN_Score': KNN_Score[0],
  #            'LIGHTGBM_Label': LIGHTGBM_Label[0],
  #            'LIGHTGBM_Score': LIGHTGBM_Score[0],
              'GBC_Label': GBC_Label[0],
              'GBC_Score': GBC_Score[0],              
             }
    
    result_output = result_output.append(result, ignore_index = True)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
CPU times: user 1h 12min 7s, sys: 15min 18s, total: 1h 27min 26s
Wall time: 1h 6min 34s


In [14]:
result_output.to_csv('Back_Test_Final_Result_12032021.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Final_Result_12032021.csv')

In [15]:
result_output = result_output[result_output['Current_Market_Fit'] > 20]
result_output = result_output.reset_index()
del result_output['index']

In [16]:
result_output.shape

(2825, 7)

In [17]:
result_output.to_csv('Back_Test_Final_Result_12032021.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Final_Result_12032021.csv')

In [18]:
#result_output['LR_Prediction'] = result_output['Current_Market'] - result_output['LR_Label']
#result_output['ET_Prediction'] = result_output['Current_Market'] - result_output['ET_Label']
result_output['KNN_Prediction'] = result_output['Current_Market'] - result_output['KNN_Label']
#result_output['LIGHTGBM_Prediction'] = result_output['Current_Market'] - result_output['LIGHTGBM_Label']
result_output['GBC_Prediction'] = result_output['Current_Market'] - result_output['GBC_Label']

In [19]:
result_output.head()

,Candle_No,Current_Market_Fit,Current_Market,KNN_Label,KNN_Score,GBC_Label,GBC_Score,KNN_Prediction,GBC_Prediction
0,24.0,85.0,0.0,0.0,1.0000,0.0,0.5477,0.0,0.0
1,653.0,58.0,1.0,1.0,0.6691,0.0,0.5457,0.0,1.0
2,9425.0,33.0,0.0,0.0,1.0000,0.0,0.5408,0.0,0.0
3,9431.0,92.0,0.0,0.0,1.0000,1.0,0.5037,0.0,-1.0
4,4751.0,74.0,0.0,0.0,1.0000,1.0,0.5294,0.0,-1.0


In [20]:
KNN_Result = pd.DataFrame(result_output['KNN_Prediction'].value_counts())
KNN_Result["Score"] = (KNN_Result['KNN_Prediction'] * 100 / len(result_output)).round(2)

In [21]:
GBC_Result = pd.DataFrame(result_output['GBC_Prediction'].value_counts())
GBC_Result["Score"] = (GBC_Result['GBC_Prediction'] * 100 / len(result_output)).round(2)

In [23]:
result = pd.concat([
#    LR_Result,
#    ET_Result,
    KNN_Result,
    GBC_Result,
], axis=1)
result

,KNN_Prediction,Score,GBC_Prediction,Score
-1.0,316,11.19,478,16.92
0.0,2195,77.70,1860,65.84
1.0,314,11.12,487,17.24


In [24]:
result_output.to_csv('Back_Test_Final_Result.csv', header = True, index = False)